In [1]:
import numpy as np
import scipy
import utils
from time import time
from utils import QR_Factorization, EVD, SVD, Bidiagonal_fastMult

%load_ext autoreload
%autoreload 2

# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [2]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


## Phase-II

In [4]:
m = n = 1024
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

In [5]:
B, _, _ = SVD.svd_phaseI(A)

In [6]:
numpy_mul_begin = time()
for i in range(1000):
    A@B
numpy_mul_end = time()
print("{:.4f}s".format(numpy_mul_end - numpy_mul_begin))

19.4035s


In [7]:
fastMul_begin = time()
for i in range(1000):
    Bidiagonal_fastMult.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
print("{:.4f}s".format(fastMul_end - fastMul_begin))

26.7734s


**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [111]:
m = 100
n = 100
A = np.random.rand(m,n)
A[10:50] = A[10:50] * 99

In [114]:
# U, S, Vt = SVD.svd(A, phaseII='A')
# U, S, Vt = SVD.svd(A, phaseII='B1')
U, S, Vt = SVD.svd(A, phaseII='B2')
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

phaseI: 0.0159s
[10391]
phaseII: 1.8110s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
98.34 %


In [15]:
S

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

**Accuracy Test:**

In [10]:
acc = 1e-12
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

Percentage of entrices successfully recovered by SVD with accuracy: 1e-12
100.0 %


**Scipy SVD**

In [11]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


## Test for p2

In [12]:
m = n = 500
k = 200
A = np.array(np.diag([1/2]*n, 0)+np.diag([3/8]*(n-1), 1)+np.diag([1/8]*(n-1), -1))

A = np.linalg.matrix_power(A, k)
U, S, Vt = SVD.svd(A, phaseII="A")
# U, S, Vt = scipy.linalg.svd(A)
acc = 1e-8
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

phaseI: 1.0642s
phaseII: 0.2348s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
